In [394]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta



In [396]:
file = '20200422.csv'
df = pd.read_csv(file, low_memory=False) 

target_date_str = re.sub(r'(\d+)\.csv', '\\1', file)
target_date = datetime.strptime(target_date_str, '%Y%m%d')

one_week_ago = target_date - timedelta(days=7)
two_weeks_ago = target_date - timedelta(days=14)
alive_date = '9999-99-99'

print('evaluating for {}'.format(target_date))
print('one week ago! {}'.format(one_week_ago))
print('two week ago! {}'.format(two_weeks_ago))

df = df.astype({'RESULTADO': 'int32'})

df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'], format="%Y-%m-%d")

df['RESULTADO_FACTOR'] = 'Desconocido'
df.loc[df['RESULTADO'] == 1, 'RESULTADO_FACTOR'] = 'Positivos'
df.loc[df['RESULTADO'] == 2, 'RESULTADO_FACTOR'] = 'Negativos'
df.loc[df['RESULTADO'] == 3, 'RESULTADO_FACTOR'] = 'Sospechosos'

df['IS_PATIENT'] = 1
df['ALL_TIME'] = True
df['LAST_7_DAYS'] = df.FECHA_SINTOMAS > one_week_ago
df['LAST_14_DAYS'] = df.FECHA_SINTOMAS > two_weeks_ago
df['HAS_DIED'] = df.FECHA_DEF != alive_date


states = pd.read_csv('../../../otros/estados.csv')
states = states.rename({'Clave': 'ENTIDAD_RES', 'ISO_3':'ENTIDAD_RES_FACTOR'}, axis=1)[['ENTIDAD_RES', 'ENTIDAD_RES_FACTOR']]
df['ENTIDAD_RES'] = df['ENTIDAD_UM']
df = df.merge(states, on='ENTIDAD_RES', how='left')


evaluating for 2020-04-22 00:00:00
one week ago! 2020-04-15 00:00:00
two week ago! 2020-04-08 00:00:00


In [397]:
def modify_pivot_df(df):
    df.columns = df.columns.get_level_values(2)
    df = df.reset_index()
    df.index.name = None

    df['Sospechosos'].fillna(0, inplace=True)
    df['Negativos'].fillna(0, inplace=True)
    df['Positivos'].fillna(0, inplace=True)    
    df = df.astype({
        'Sospechosos': 'int32', 
        'Negativos': 'int32', 
        'Positivos': 'int32'
    })

    df = df.rename(
        columns={
            'ENTIDAD_RES_FACTOR': 'Estado'
        })
    
    return df


def get_main_series(df, value='ALL_TIME', prefix='', suffix=''):
    my_df = pd.pivot_table(
        df, 
        values=[value],  
        index=['ENTIDAD_RES_FACTOR',  'RESULTADO_FACTOR'],
        columns=['IS_PATIENT'], aggfunc=np.sum
    ).unstack()

    my_df = modify_pivot_df(my_df)
    
    my_df = my_df.rename(
        columns={
            'Positivos': '{}Positivos{}'.format(prefix, suffix),
            'Sospechosos': '{}Sospechosos{}'.format(prefix, suffix),
            'Negativos': '{}Negativos{}'.format(prefix, suffix)
        })

    return my_df[['Estado', 
                  '{}Positivos{}'.format(prefix, suffix), 
                  '{}Sospechosos{}'.format(prefix, suffix), 
                  '{}Negativos{}'.format(prefix, suffix)]]


main = get_main_series(df)
main['Fecha'] = target_date.strftime('%Y-%m-%d')
main = main[['Fecha', 'Estado', 'Positivos', 'Sospechosos', 'Negativos']]

deaths = get_main_series(df[df['HAS_DIED'] == True], prefix='Defunciones_')
symptoms_last_14_days = get_main_series(df, value='LAST_14_DAYS', suffix='_Sintomas_14_Dias')
symptoms_last_7_days = get_main_series(df, value='LAST_7_DAYS', suffix='_Sintomas_7_Dias')

summary_df = main.merge(deaths, on='Estado', how='left')
summary_df = summary_df.merge(symptoms_last_14_days, on='Estado', how='left')
summary_df = summary_df.merge(symptoms_last_7_days, on='Estado', how='left')
summary_df

summary_df.to_csv('federal_{}'.format(file), index=False)  